### Web Scrapping

In [1]:
# !pip install --upgrade pip
# !pip install beautifulsoup4 --upgrade
# pip install lxml
# pip install html5lib

In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from json2html import *
import re
import requests
from urllib.request import Request, urlopen
from urllib import parse
import glob

#### External HTML File From Browser Element Section 

In [1085]:
lang = "indonesian"  # indonesian, turkish, russian, english, urdu, hindi, bangla
surah = "an nahl"  # al faatiha, al baqara, aal i imraan ...
folder = f"{surah}-16"  # al faatiha-1, al baqara-2, aal i imraan-3 ...
file = f"{surah.lower()}_{lang.lower()}"

In [1086]:
HTMLFile = open(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Data/{folder.lower().capitalize()}/{file.lower()}.html", "r") # ayahs-block

In [1087]:
content = HTMLFile.read()

In [1088]:
soup = BeautifulSoup(content, 'lxml')

In [1089]:
#print(soup.prettify())

In [1090]:
ayah_word = soup.findAll('span', attrs={"class":"single-word"})
#ayah_word

In [1091]:
word_list = []
for word in ayah_word:
    var1 = word.find('span', attrs={"class":"word-arabic word-arabic-uthmani-hafs"})
    #try:
    #    var1_list = re.findall(r'''(?<=>)(.*)(?=</span>)''', var1)
    #    arabic=" ".join(var1_list)
    #except:
    #    pass
    #arabic = var1.get_text()
    arabic = var1.string
    var2 = word.find('span', attrs={"class":"word-transliteration"})
    transliterate = var2.string
    var3 = word.find('span', attrs={"class":"word-translation"})
    translate = var3.string
    word_list.append((arabic, transliterate, translate))

In [1092]:
word_list

[('أَتَىٰٓ', 'ata', 'telah'),
 ('أَمۡرُ', 'amru', 'ketetapan'),
 ('ٱللَّهِ', 'al-lahi', 'Allah'),
 ('فَلَا', 'fala', 'maka janganlah'),
 ('تَسۡتَعۡجِلُوهُۚ', "tasta'jiluhu", 'kamu minta disegerakannya'),
 ('سُبۡحَٰنَهُۥ', "sub'hanahu", 'Maha Suci Dia'),
 ('وَتَعَٰلَىٰ', "wata'ala", 'dan Maha Tinggi'),
 ('عَمَّا', 'amma', 'dari apa'),
 ('يُشۡرِكُونَ', "yush'rikuna", 'mereka persekutukan'),
 ('يُنَزِّلُ', 'yunazzilu', 'Dia menurunkan'),
 ('ٱلۡمَلَـٰٓئِكَةَ', 'al-malaikata', 'Malaikat'),
 ('بِٱلرُّوحِ', 'bil-ruhi', 'dengan ruh'),
 ('مِنۡ', 'min', 'dari'),
 ('أَمۡرِهِۦ', 'amrihi', 'perintahnya'),
 ('عَلَىٰ', 'ala', 'atas'),
 ('مَن', 'man', 'siapa'),
 ('يَشَآءُ', 'yashau', 'Dia kehendaki'),
 ('مِنۡ', 'min', 'dari'),
 ('عِبَادِهِۦٓ', 'ibadihi', 'hamba-hambaNya'),
 ('أَنۡ', 'an', 'hendaklah'),
 ('أَنذِرُوٓاْ', 'andhiru', 'kamu beri peringatan'),
 ('أَنَّهُۥ', 'annahu', 'bahwasanya Dia'),
 ('لَآ', 'la', 'tidak ada'),
 ('إِلَٰهَ', 'ilaha', 'Tuhan'),
 ('إِلَّآ', 'illa', 'melainkan'),
 ('أَنَا

In [1093]:
df_word_list = pd.DataFrame(word_list)
df_word_list.rename(columns={0:"arabic", 1:"transliterate", 2:f"translate_{lang.lower()}"}, inplace=True)
# df_word_list = df_word_list.iloc[3:,] for all body element
df_word_list

,arabic,transliterate,translate_indonesian
0,أَتَىٰٓ,ata,telah
1,أَمۡرُ,amru,ketetapan
2,ٱللَّهِ,al-lahi,Allah
3,فَلَا,fala,maka janganlah
4,تَسۡتَعۡجِلُوهُۚ,tasta'jiluhu,kamu minta disegerakannya
...,...,...,...
1839,ٱلَّذِينَ,alladhina,orang-orang yang
1840,ٱتَّقَواْ,ittaqaw,(mereka) bertakwa
1841,وَّٱلَّذِينَ,wa-alladhina,dan orang-orang yang
1842,هُم,hum,mereka


In [1094]:
df_word_list.to_excel(f"{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Words_{lang.lower().capitalize()}.xlsx", index=False)

#### Merge Word All Translate

In [1095]:
all_trans_file = glob.glob(f"{surah.lower().capitalize()}*")
all_trans_file

['An nahl_Arabic_Transliterate_Translate_Words_Bangla.xlsx',
 'An nahl_Arabic_Transliterate_Translate_Words_Hindi.xlsx',
 'An nahl_Arabic_Transliterate_Translate_Words_Urdu.xlsx',
 'An nahl_Arabic_Transliterate_Translate_Words_English.xlsx',
 'An nahl_Arabic_Transliterate_Translate_Words_Russian.xlsx',
 'An nahl_Arabic_Transliterate_Translate_Words_Turkish.xlsx',
 'An nahl_Arabic_Transliterate_Translate_Words_Indonesian.xlsx']

In [1096]:
df_master = pd.read_excel(all_trans_file[0])
for i in all_trans_file[1:]:
    df_var = pd.read_excel(f"{i}")
    #df_master = pd.merge(df_master, df_var, how="left", on=["arabic","transliterate"])
    df_master = pd.concat([df_master,df_var.iloc[:,2]], axis=1)
    #df_master = df_master.join(df_var, how="left", lsuffix=["arabic","transliterate"],rsuffix=["arabic","transliterate"])
    #df_master.drop_duplicates(inplace=True)
    df_master.reset_index(drop=True, inplace=True)
df_master = df_master[["arabic","transliterate","translate_english","translate_urdu","translate_hindi","translate_indonesian","translate_bangla","translate_turkish","translate_russian"]]
df_master

,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,أَتَىٰٓ,ata,Will come,آگیا,आ पहुँचा,telah,এসেছে,geldi,Пришло [придёт]
1,أَمۡرُ,amru,the command,فیصلہ,हुक्म,ketetapan,আদেশ,emri,повеление
2,ٱللَّهِ,al-lahi,of Allah,اللہ کا,अल्लाह का,Allah,আল্লাহ্‌র,Allah'ın,"Аллаха,"
3,فَلَا,fala,so (do) not,پس نہ,पस ना,maka janganlah,সুতরাং না,artık,так не
4,تَسۡتَعۡجِلُوهُۚ,tasta'jiluhu,(be) impatient for it,جلدی مانگو اس کو,तुम जल्दी माँगो उसे,kamu minta disegerakannya,তা তোমরা তাড়াহুড়া করতে চেয়ো,onu acele istemeyin,торопите же его!
...,...,...,...,...,...,...,...,...,...
1839,ٱلَّذِينَ,alladhina,those who,ان لوگوں کے,उनके जो,orang-orang yang,যারা,kimselerle,"теми, которые"
1840,ٱتَّقَواْ,ittaqaw,fear (Him),جنہوں نے تقوی کیا,तक़वा करें,(mereka) bertakwa,তাকওয়া অবলম্বন করে,korunan(larla),остерегаются
1841,وَّٱلَّذِينَ,wa-alladhina,and those who,اور ان لوگوں کے,और वो लोग जो,dan orang-orang yang,এবং যারা,ve kimselerle,"и теми, которые –"
1842,هُم,hum,[they],وہ,वो,mereka,তারা,onlar,они


In [1097]:
df_master.to_excel(f"{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx", index=False)

#### Merge Surah All Translate

In [164]:
def add_info(df,surah_name, surah_num, index):
    for i in range(len(df)):
        df["surah latin"] = surah_name
        df["surah"] = surah_num
        df.loc[i,"index"] = i+index
    return df

In [159]:
df_faatir = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Faatir-35/Faatir_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_faatir

,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,ٱلۡحَمۡدُ,al-hamdu,All praises,سب تعریف,सब तारीफ़,segala puji,সকল প্রশংসা,hamd olsun,Хвала
1,لِلَّهِ,lillahi,(be) to Allah,اللہ کے لیے ہے,अल्लाह के लिए है,bagi Allah,আল্লাহরই জন্যে,Allah'a,"Аллаху,"
2,فَاطِرِ,fatiri,Creator,پیدا کرنے والا ہے,पैदा करने वाला है,pencipta,(যিনি) স্রষ্টা,yoktan var eden,Творцу
3,ٱلسَّمَٰوَٰتِ,al-samawati,(of) the heavens,آسمانوں کا,आसमानों,langit(jamak),আকাশের,gökleri,небес
4,وَٱلۡأَرۡضِ,wal-ardi,and the earth,اور زمین کا,और ज़मीन का,dan bumi,ও পৃথিবীর,ve yeri,"и земли,"
...,...,...,...,...,...,...,...,...,...
770,فَإِنَّ,fa-inna,then indeed,تو بیشک,तो बेशक,maka sesungguh-Nya,নিশ্চযই় তখন,kuşkusuz,"то, поистине,"
771,ٱللَّهَ,al-laha,Allah,اللہ تعالیٰ,अल्लाह,Allah,আল্লাহ,Allah,Аллах
772,كَانَ,kana,is,ہے,है,adalah,হবেন,kullarını,является
773,بِعِبَادِهِۦ,bi'ibadihi,of His slaves,اپنے بندوں کے ساتھ,अपने बन्दों को,dengan hamba-hamba-Nya,তাঁর দাসদের সম্পর্কে,*,Своих рабов


In [166]:
df_faatir = add_info(df_faatir, "Fatir", 35, 56320)
df_faatir = df_faatir.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_faatir

,surah,surah latin,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian,index
0,35,Fatir,ٱلۡحَمۡدُ,al-hamdu,All praises,سب تعریف,सब तारीफ़,segala puji,সকল প্রশংসা,hamd olsun,Хвала,56320.0
1,35,Fatir,لِلَّهِ,lillahi,(be) to Allah,اللہ کے لیے ہے,अल्लाह के लिए है,bagi Allah,আল্লাহরই জন্যে,Allah'a,"Аллаху,",56321.0
2,35,Fatir,فَاطِرِ,fatiri,Creator,پیدا کرنے والا ہے,पैदा करने वाला है,pencipta,(যিনি) স্রষ্টা,yoktan var eden,Творцу,56322.0
3,35,Fatir,ٱلسَّمَٰوَٰتِ,al-samawati,(of) the heavens,آسمانوں کا,आसमानों,langit(jamak),আকাশের,gökleri,небес,56323.0
4,35,Fatir,وَٱلۡأَرۡضِ,wal-ardi,and the earth,اور زمین کا,और ज़मीन का,dan bumi,ও পৃথিবীর,ve yeri,"и земли,",56324.0
...,...,...,...,...,...,...,...,...,...,...,...,...
770,35,Fatir,فَإِنَّ,fa-inna,then indeed,تو بیشک,तो बेशक,maka sesungguh-Nya,নিশ্চযই় তখন,kuşkusuz,"то, поистине,",57090.0
771,35,Fatir,ٱللَّهَ,al-laha,Allah,اللہ تعالیٰ,अल्लाह,Allah,আল্লাহ,Allah,Аллах,57091.0
772,35,Fatir,كَانَ,kana,is,ہے,है,adalah,হবেন,kullarını,является,57092.0
773,35,Fatir,بِعِبَادِهِۦ,bi'ibadihi,of His slaves,اپنے بندوں کے ساتھ,अपने बन्दों को,dengan hamba-hamba-Nya,তাঁর দাসদের সম্পর্কে,*,Своих рабов,57093.0


In [160]:
df_yaseen = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Yaseen-36/Yaseen_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_yaseen

,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,يسٓ,ya-seen,Ya Sin,یس,ي س,Yâ­Sîn,ইয়া-সীন।,Yâ Sîn,Йа син
1,وَٱلۡقُرۡءَانِ,wal-qur'ani,By the Quran,قسم ہے قرآن,क़सम है क़ुरआन,demi Al Qur'an,শপথ কুরআনের (যা),Kur'an'a andolsun,клянусь Кораном
2,ٱلۡحَكِيمِ,al-hakimi,the Wise,حکیم کی,हिकमत वाले की,penuh hikmat,জ্ঞানময় (বিজ্ঞানময়)।,hikmetli,мудрым
3,إِنَّكَ,innaka,Indeed you,بیشک آپ,बेशक आप,sesungguhnya kamu,তুমি নিশ্চয়ই,kuşkusuz sen,"(что) поистине, ты"
4,لَمِنَ,lamina,(are) among,البتہ سے ہیں,*,sungguh dari (salah seorang),অন্তর্ভুক্ত অবশ্যই,gönderilmiş elçilerdensin,"однозначно, из"
...,...,...,...,...,...,...,...,...,...
720,مَلَكُوتُ,malakutu,is (the) dominion,بادشاہت ہے,बादशाहत है,kekuasaan,সার্বভৌম ক্ষমতা,hükümranlığı,владычество
721,كُلِّ,kulli,(of) all,ہر,हर,segala,সব,her,над всякой
722,شَيۡءٖ,shayin,things,چیز کی,चीज़ की,sesuatu,জিনিসের,şeyin,вещью
723,وَإِلَيۡهِ,wa-ilayhi,and to Him,اور اسی کی طرف,और तरफ़ उसीके,dan kepadaNya,এবং তাঁরই দিকে,ve O'na,и к Нему


In [167]:
df_yaseen = add_info(df_yaseen, "Ya Seen", 36, 57102)
df_yaseen = df_yaseen.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_yaseen

,surah,surah latin,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian,index
0,36,Ya Seen,يسٓ,ya-seen,Ya Sin,یس,ي س,Yâ­Sîn,ইয়া-সীন।,Yâ Sîn,Йа син,57102.0
1,36,Ya Seen,وَٱلۡقُرۡءَانِ,wal-qur'ani,By the Quran,قسم ہے قرآن,क़सम है क़ुरआन,demi Al Qur'an,শপথ কুরআনের (যা),Kur'an'a andolsun,клянусь Кораном,57103.0
2,36,Ya Seen,ٱلۡحَكِيمِ,al-hakimi,the Wise,حکیم کی,हिकमत वाले की,penuh hikmat,জ্ঞানময় (বিজ্ঞানময়)।,hikmetli,мудрым,57104.0
3,36,Ya Seen,إِنَّكَ,innaka,Indeed you,بیشک آپ,बेशक आप,sesungguhnya kamu,তুমি নিশ্চয়ই,kuşkusuz sen,"(что) поистине, ты",57105.0
4,36,Ya Seen,لَمِنَ,lamina,(are) among,البتہ سے ہیں,*,sungguh dari (salah seorang),অন্তর্ভুক্ত অবশ্যই,gönderilmiş elçilerdensin,"однозначно, из",57106.0
...,...,...,...,...,...,...,...,...,...,...,...,...
720,36,Ya Seen,مَلَكُوتُ,malakutu,is (the) dominion,بادشاہت ہے,बादशाहत है,kekuasaan,সার্বভৌম ক্ষমতা,hükümranlığı,владычество,57822.0
721,36,Ya Seen,كُلِّ,kulli,(of) all,ہر,हर,segala,সব,her,над всякой,57823.0
722,36,Ya Seen,شَيۡءٖ,shayin,things,چیز کی,चीज़ की,sesuatu,জিনিসের,şeyin,вещью,57824.0
723,36,Ya Seen,وَإِلَيۡهِ,wa-ilayhi,and to Him,اور اسی کی طرف,और तरफ़ उसीके,dan kepadaNya,এবং তাঁরই দিকে,ve O'na,и к Нему,57825.0


In [161]:
df_saaffaat = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/As Saaffaat-37/As saaffaat_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_saaffaat

,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,وَٱلصَّـٰٓفَّـٰتِ,wal-safati,By those lined,قسم ہے صفیں باندھنے والوں کی,क़सम है सफ़ बाँधने वालों की,demi yang berbaris,শপথ সারি বেঁধে দাঁড়ায় যারা,andolsun,Клянусь стоящими рядами
1,صَفّٗا,saffan,(in) rows,صف در صف,ख़ूब सफ़ बाँधना,barisan teratur,সারি সারি,sıra sıra dizilenlere,"по рядам,"
2,فَٱلزَّـٰجِرَٰتِ,fal-zajirati,And those who drive,پھر ڈانٹ دینے والوں کی,फिर डाँटने वालों की,maka,অতঃপর (শপথ) ধমকদানকারীদের,ve sürenlere,и гонящими
3,زَجۡرٗا,zajran,strongly,سخت ڈانٹنا,सख़्त डाँटना,sebenar-benar larangan,সজোরে,bağırıp,"упорно,"
4,فَٱلتَّـٰلِيَٰتِ,fal-taliyati,And those who recite,پھر تلاوت کرنے والوں کی,फिर तिलावत करने वालों की,maka,অতঃপর (শপথ তাদের) পাঠকদের,ve okuyanlara,и читающими
...,...,...,...,...,...,...,...,...,...
855,ٱلۡمُرۡسَلِينَ,al-mur'salina,the Messengers,سب رسولوں,सब रसूलों पर,para Rasul,রাসূলদের,gönderilen elçiler,посланными!
856,وَٱلۡحَمۡدُ,wal-hamdu,And all praise,اور سب تعریف,और सब तारीफ़,dan segala puji,এবং সমস্ত প্রশংসা,ve hamd olsun,И хвала
857,لِلَّهِ,lillahi,(be) to Allah,اللہ,अल्लाह के लिए है,bagi Allah,আল্লাহরই,Allah'a,"Аллаху,"
858,رَبِّ,rabbi,(the) Lord,رب,जो रब है,Tuhan,(যিনি) রব,Rabbi,Господу


In [168]:
df_saaffaat = add_info(df_saaffaat, "As-Saffat", 37, 57836)
df_saaffaat = df_saaffaat.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_saaffaat

,surah,surah latin,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian,index
0,37,As-Saffat,وَٱلصَّـٰٓفَّـٰتِ,wal-safati,By those lined,قسم ہے صفیں باندھنے والوں کی,क़सम है सफ़ बाँधने वालों की,demi yang berbaris,শপথ সারি বেঁধে দাঁড়ায় যারা,andolsun,Клянусь стоящими рядами,57836.0
1,37,As-Saffat,صَفّٗا,saffan,(in) rows,صف در صف,ख़ूब सफ़ बाँधना,barisan teratur,সারি সারি,sıra sıra dizilenlere,"по рядам,",57837.0
2,37,As-Saffat,فَٱلزَّـٰجِرَٰتِ,fal-zajirati,And those who drive,پھر ڈانٹ دینے والوں کی,फिर डाँटने वालों की,maka,অতঃপর (শপথ) ধমকদানকারীদের,ve sürenlere,и гонящими,57838.0
3,37,As-Saffat,زَجۡرٗا,zajran,strongly,سخت ڈانٹنا,सख़्त डाँटना,sebenar-benar larangan,সজোরে,bağırıp,"упорно,",57839.0
4,37,As-Saffat,فَٱلتَّـٰلِيَٰتِ,fal-taliyati,And those who recite,پھر تلاوت کرنے والوں کی,फिर तिलावत करने वालों की,maka,অতঃপর (শপথ তাদের) পাঠকদের,ve okuyanlara,и читающими,57840.0
...,...,...,...,...,...,...,...,...,...,...,...,...
855,37,As-Saffat,ٱلۡمُرۡسَلِينَ,al-mur'salina,the Messengers,سب رسولوں,सब रसूलों पर,para Rasul,রাসূলদের,gönderilen elçiler,посланными!,58691.0
856,37,As-Saffat,وَٱلۡحَمۡدُ,wal-hamdu,And all praise,اور سب تعریف,और सब तारीफ़,dan segala puji,এবং সমস্ত প্রশংসা,ve hamd olsun,И хвала,58692.0
857,37,As-Saffat,لِلَّهِ,lillahi,(be) to Allah,اللہ,अल्लाह के लिए है,bagi Allah,আল্লাহরই,Allah'a,"Аллаху,",58693.0
858,37,As-Saffat,رَبِّ,rabbi,(the) Lord,رب,जो रब है,Tuhan,(যিনি) রব,Rabbi,Господу,58694.0


In [162]:
df_saad = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Saad-38/Saad_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_saad

,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,صٓۚ,sad,Saad,ص,ص,shad,সাদ,Sâd,Сад.
1,وَٱلۡقُرۡءَانِ,wal-qur'ani,By the Quran,قسم ہے قرآن کی،,क़सम है क़ुरआन,demi Al Quran,কোরআনের শপথ,Kur'an'a andolsun,"Клянусь Кораном,"
2,ذِي,dhi,full (of) reminder,والے,*,mempunyai,পূর্ণ,sahibi,обладающим
3,ٱلذِّكۡرِ,al-dhik'ri,full (of) reminder,نصحیت,नसीहत वाले की,peringatan,উপদেশ,şan şeref',напоминанием
4,بَلِ,bali,Nay,بلکہ,बल्कि,bahkan,কিন্তু,doğrusu,Однако
...,...,...,...,...,...,...,...,...,...
728,لِّلۡعَٰلَمِينَ,lil'alamina,to the worlds,جہان والوں کے لئے,तमाम जहान वालों के लिए,bagi semesta alam,বিশ্ববাসীদের জন্যে,bütün alemlere,"для миров,"
729,وَلَتَعۡلَمُنَّ,walata'lamunna,And surely you will know,اور البتہ تم ضرور جان لو گے,और अलबत्ता तुम ज़रूर जान लोगे,dan sungguh kamu akan mengetahui,এবং তোমরা অবশ্যই জানবে,gayet iyi bileceksiniz,И непременно вы узнаете
730,نَبَأَهُۥ,naba-ahu,its information,خبر اس کی,ख़बर उसकी,beritanya,তার খবর,onun haberini,весть (истинности) его
731,بَعۡدَ,ba'da,after,بعد,बाद,sesudah,পরেই,sonra,после


In [169]:
df_saad = add_info(df_saad, "Sad", 38, 58705)
df_saad = df_saad.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_saad

,surah,surah latin,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian,index
0,38,Sad,صٓۚ,sad,Saad,ص,ص,shad,সাদ,Sâd,Сад.,58705.0
1,38,Sad,وَٱلۡقُرۡءَانِ,wal-qur'ani,By the Quran,قسم ہے قرآن کی،,क़सम है क़ुरआन,demi Al Quran,কোরআনের শপথ,Kur'an'a andolsun,"Клянусь Кораном,",58706.0
2,38,Sad,ذِي,dhi,full (of) reminder,والے,*,mempunyai,পূর্ণ,sahibi,обладающим,58707.0
3,38,Sad,ٱلذِّكۡرِ,al-dhik'ri,full (of) reminder,نصحیت,नसीहत वाले की,peringatan,উপদেশ,şan şeref',напоминанием,58708.0
4,38,Sad,بَلِ,bali,Nay,بلکہ,बल्कि,bahkan,কিন্তু,doğrusu,Однако,58709.0
...,...,...,...,...,...,...,...,...,...,...,...,...
728,38,Sad,لِّلۡعَٰلَمِينَ,lil'alamina,to the worlds,جہان والوں کے لئے,तमाम जहान वालों के लिए,bagi semesta alam,বিশ্ববাসীদের জন্যে,bütün alemlere,"для миров,",59433.0
729,38,Sad,وَلَتَعۡلَمُنَّ,walata'lamunna,And surely you will know,اور البتہ تم ضرور جان لو گے,और अलबत्ता तुम ज़रूर जान लोगे,dan sungguh kamu akan mengetahui,এবং তোমরা অবশ্যই জানবে,gayet iyi bileceksiniz,И непременно вы узнаете,59434.0
730,38,Sad,نَبَأَهُۥ,naba-ahu,its information,خبر اس کی,ख़बर उसकी,beritanya,তার খবর,onun haberini,весть (истинности) его,59435.0
731,38,Sad,بَعۡدَ,ba'da,after,بعد,बाद,sesudah,পরেই,sonra,после,59436.0


In [163]:
df_zumar = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Az Zumar-39/Az zumar_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_zumar

,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,تَنزِيلُ,tanzilu,(The) revelation,نازل کرنا ہے,नाज़िल करना हैं,turunnya,অবতীর্ণ করা,indirilmesi,Ниспослание
1,ٱلۡكِتَٰبِ,al-kitabi,(of) the Book,کتاب کا,किताब का,Kitab,(এই) কিতাব,Kitabını,Книги
2,مِنَ,mina,(is) from,سے,*,dari,পক্ষ হ'তে,tarafındandır,от
3,ٱللَّهِ,al-lahi,Allah,اللہ کی طرف,अल्लाह की तरफ़ से,Allah,আল্লাহর,Allah,"Аллаха,"
4,ٱلۡعَزِيزِ,al-'azizi,the All-Mighty,جو غالب ہے،,जो बहुत ज़बरदस्त है,Maha Perkasa,( যিনি) পরাক্রমশালী,aziz,"Величественного,"
...,...,...,...,...,...,...,...,...,...
1167,وَقِيلَ,waqila,and it will be said,اور کہہ دیا جائے گا,और कह दिया जाएगा,dan dikatakan,এবং বলা হবে,ve denilir,и будет сказано:
1168,ٱلۡحَمۡدُ,al-hamdu,All praise be,سب تعریف,सब तारीफ़,segala puji,"""সকল প্রশংসা",Hamd,«Хвала
1169,لِلَّهِ,lillahi,to Allah,اللہ کے لیے ہے,अल्लाह के लिए है,bagi Allah,আল্লাহর জন্যে,Allah'a'dır,Аллаху
1170,رَبِّ,rabbi,(the) Lord,جو رب ہے,जो रब है,Tuhan,(যিনি) রব,Rabbi,Господу


In [170]:
df_zumar = add_info(df_zumar, "Az-Zumar", 39, 59444)
df_zumar = df_zumar.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_zumar

,surah,surah latin,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian,index
0,39,Az-Zumar,تَنزِيلُ,tanzilu,(The) revelation,نازل کرنا ہے,नाज़िल करना हैं,turunnya,অবতীর্ণ করা,indirilmesi,Ниспослание,59444.0
1,39,Az-Zumar,ٱلۡكِتَٰبِ,al-kitabi,(of) the Book,کتاب کا,किताब का,Kitab,(এই) কিতাব,Kitabını,Книги,59445.0
2,39,Az-Zumar,مِنَ,mina,(is) from,سے,*,dari,পক্ষ হ'তে,tarafındandır,от,59446.0
3,39,Az-Zumar,ٱللَّهِ,al-lahi,Allah,اللہ کی طرف,अल्लाह की तरफ़ से,Allah,আল্লাহর,Allah,"Аллаха,",59447.0
4,39,Az-Zumar,ٱلۡعَزِيزِ,al-'azizi,the All-Mighty,جو غالب ہے،,जो बहुत ज़बरदस्त है,Maha Perkasa,( যিনি) পরাক্রমশালী,aziz,"Величественного,",59448.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1167,39,Az-Zumar,وَقِيلَ,waqila,and it will be said,اور کہہ دیا جائے گا,और कह दिया जाएगा,dan dikatakan,এবং বলা হবে,ve denilir,и будет сказано:,60611.0
1168,39,Az-Zumar,ٱلۡحَمۡدُ,al-hamdu,All praise be,سب تعریف,सब तारीफ़,segala puji,"""সকল প্রশংসা",Hamd,«Хвала,60612.0
1169,39,Az-Zumar,لِلَّهِ,lillahi,to Allah,اللہ کے لیے ہے,अल्लाह के लिए है,bagi Allah,আল্লাহর জন্যে,Allah'a'dır,Аллаху,60613.0
1170,39,Az-Zumar,رَبِّ,rabbi,(the) Lord,جو رب ہے,जो रब है,Tuhan,(যিনি) রব,Rabbi,Господу,60614.0


In [174]:
df_surah_merge = pd.concat([df_faatir,df_yaseen,df_saaffaat,df_saad,df_zumar], axis=0)
df_surah_merge

,surah,surah latin,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian,index
0,35,Fatir,ٱلۡحَمۡدُ,al-hamdu,All praises,سب تعریف,सब तारीफ़,segala puji,সকল প্রশংসা,hamd olsun,Хвала,56320.0
1,35,Fatir,لِلَّهِ,lillahi,(be) to Allah,اللہ کے لیے ہے,अल्लाह के लिए है,bagi Allah,আল্লাহরই জন্যে,Allah'a,"Аллаху,",56321.0
2,35,Fatir,فَاطِرِ,fatiri,Creator,پیدا کرنے والا ہے,पैदा करने वाला है,pencipta,(যিনি) স্রষ্টা,yoktan var eden,Творцу,56322.0
3,35,Fatir,ٱلسَّمَٰوَٰتِ,al-samawati,(of) the heavens,آسمانوں کا,आसमानों,langit(jamak),আকাশের,gökleri,небес,56323.0
4,35,Fatir,وَٱلۡأَرۡضِ,wal-ardi,and the earth,اور زمین کا,और ज़मीन का,dan bumi,ও পৃথিবীর,ve yeri,"и земли,",56324.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1167,39,Az-Zumar,وَقِيلَ,waqila,and it will be said,اور کہہ دیا جائے گا,और कह दिया जाएगा,dan dikatakan,এবং বলা হবে,ve denilir,и будет сказано:,60611.0
1168,39,Az-Zumar,ٱلۡحَمۡدُ,al-hamdu,All praise be,سب تعریف,सब तारीफ़,segala puji,"""সকল প্রশংসা",Hamd,«Хвала,60612.0
1169,39,Az-Zumar,لِلَّهِ,lillahi,to Allah,اللہ کے لیے ہے,अल्लाह के लिए है,bagi Allah,আল্লাহর জন্যে,Allah'a'dır,Аллаху,60613.0
1170,39,Az-Zumar,رَبِّ,rabbi,(the) Lord,جو رب ہے,जो रब है,Tuhan,(যিনি) রব,Rabbi,Господу,60614.0


In [175]:
df_surah_merge.to_excel("Faatir_Yaseen_Saaffaat_Saad_Zumar_Surah_Translate_Concat.xlsx", index=False)